In [38]:
import numpy as np
import os
import pandas as pd
import glob
import h5py
import tables
from ast import literal_eval
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

## Merging Data

This script takes raw data, create a pandas df and saves it to disk as a hdf5 file called `data.h5`. Before running this code, make sure your file directory looks like this...

```
.
│   Data Merged.ipynb
│   ...    
│
└───Data
    │   search_grid.csv
    │   impact_score_2017.csv
    │   impact_score_2017.csv
    |   impact_score_2019_public_test.csv
    |
    └───GFS_2017
    |   │   gfs_4_20170101_0000_000.csv
    |   │   ...
    |
    └───GFS_2018
    |   │   gfs_4_20180101_0000_000.csv
    |   │   ...     
    | 
    └───GFS_2019
        │   gfs_4_20190101_0000_000.csv
        │   ...     
   

```

# Loading Provided Data

In [39]:
#### Path to all data files

# All weather
weather17 = ["./Data/GFS_2017/" + f for f in os.listdir("./Data/GFS_2017")]
weather18 = ["./Data/GFS_2018/" + f for f in os.listdir("./Data/GFS_2018")]
weather19 = ["./Data/GFS_2019/" + f for f in os.listdir("./Data/GFS_2019")]
allweather = weather17 + weather18 + weather19

# All impact
allimpact = ["./Data/" + f for f in os.listdir("./Data") if f.startswith('impact')]

# Loc2zip map
grid = "./Data/search_grid.csv"

In [40]:
### Load weather into 1 df
# Concat into a giant dataframe
weather = pd.concat(map(pd.read_csv, allweather), ignore_index=True)

In [41]:
### Load and flatten loc2zip map
loc2zip = pd.read_csv(grid, converters={"mapped_zipcodes": literal_eval})
loc2zip = loc2zip.explode("mapped_zipcodes").reset_index()
loc2zip['mapped_zipcodes'] = pd.to_numeric(loc2zip['mapped_zipcodes'])
#loc2zip.rename(columns={"mapped_zipcodes":"zip5"}, inplace=True)

In [42]:
### Load impact into 1 df
impact = pd.concat(map(pd.read_csv, allimpact), ignore_index=True).drop(["Unnamed: 0"], axis=1)

In [43]:
### Join impact and loc2zip
impact_loc = impact.join(loc2zip.set_index('mapped_zipcodes'), on='zip5').drop(["index"], axis = 1)

In [44]:
### Join impact and weather in loc and date
# Preprocessing on join columns 
impact_loc["date_key"] = pd.to_datetime(impact_loc["date_key"])
weather["Date"] = pd.to_datetime(weather["Date"], format="%Y%m%d")

merged = impact_loc.merge(weather, left_on=["date_key", "grid_lat", "grid_lon"], right_on=["Date", "lat", "lng"], how='left')
merged = merged.sort_values(by="Date")

In [45]:
#impact_loc.to_hdf("impact_scores.h5", key='df', mode='a')

## Add Empty Rows for Missing Weather Data

In [46]:
left = merged.loc[pd.isna(merged["Date"])].sort_values("date_key")

In [47]:
# Add empty rows for impact scores that don't have any weather data
missing = left.loc[left.index.repeat(4)]

In [48]:
# Add empty rows for impact scores that have less than 4 weather timestamps
complete = merged.loc[pd.notna(merged["Date"])]
g = complete.groupby(["zip5", "date_key"])
miss3 = g.filter(lambda x: len(x) == 1)
miss2 = g.filter(lambda x: len(x) == 2)
miss1 = g.filter(lambda x: len(x) == 3)

In [49]:
# missing 3 timestamps
miss3.iloc[:, 5:] = np.NaN
add3 = miss3.loc[miss3.index.repeat(3)]
print(miss3.shape)
print(add3.shape)

(637, 123)
(1911, 123)


In [50]:
# missing 2 timestamps
add2 = miss2
add2.iloc[:, 5:] = np.NaN
print(miss2.shape)
print(add2.shape)

(182, 123)
(182, 123)


In [51]:
# missing 2 timestamp
miss1_ = miss1.iloc[:, 0:5].drop_duplicates(["date_key", "zip5"])
add1 = miss1.loc[miss1_.index.repeat(1)]
add1.iloc[:, 5:] = np.NaN
print(miss1.shape)
print(add1.shape)

(1092, 123)
(364, 123)


In [52]:
# Add missing value rows to dataframe
missing = missing.append(add1)
missing = missing.append(add2)
missing = missing.append(add3)

In [53]:
# Verify that each 
data = complete.append(missing)
gg = data.groupby(["zip5", "date_key"])
print((gg.filter(lambda x: len(x) < 4)).shape)
print((gg.filter(lambda x: len(x) == 4)).shape)
print((gg.filter(lambda x: len(x) > 4).shape))

(0, 123)
(394700, 123)
(0, 123)


## Adding Features from External Data Scources

In [54]:
### Add state, region, and urban/suburban information from FAR codes dataset
data = data.sort_values(["zip5", 'date_key', 'Time'])
x = data['zip5']
# Read in Far codes
far = pd.read_excel('FARcodesZIPdata2010WithAKandHI.xlsx', sheet_name = 1)
cols = ['ZIP', 'density', 'state']
far = far[cols]
new_row = {'ZIP': 2722, 'density': 0, 'state': 'MA'}
far = far.append(new_row, ignore_index=True)

In [55]:
# Add density and state
data = data.merge(far, left_on = 'zip5', right_on = 'ZIP')
y = data['zip5']

In [56]:
# State to region dictionary
New_England_Northeast = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
Mid_Atlantic_Northeast = ['NJ', 'NY', 'PA']
East_North_Central_Midwest = ['IL', 'IN', 'MI', 'OH', 'WI'] 
West_North_Central_Midwest = ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']
South_Atlantic_South = ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'DC', 'WV']
East_South_Central_South = ['AL', 'KY', 'MS', 'TN']
West_South_Central_South = ['AR', 'LA', 'OK', 'TX']
Mountain_West = ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY']
Pacific_West = ['AK', 'CA', 'HI', 'OR', 'WA']

# Add region
data['Region'] = ['New England Northeast' if x in New_England_Northeast else
                 'Mid Atlantic Northeast' if x in Mid_Atlantic_Northeast else
                 'East North Central Midwest' if x in East_North_Central_Midwest else
                 'West North Central Midwest' if x in West_North_Central_Midwest else
                 'South Atlantic South' if x in South_Atlantic_South else
                 'East South Central South' if x in East_South_Central_South else
                 'West South Central South' if x in West_South_Central_South else
                 'Mountain West' if x in Mountain_West else
                 'Pacific West' for x in data['state']]
data = data.drop(["ZIP"], axis=1)

In [57]:
# Add suburban or urban
data['USR'] = ['Urban' if x > 3000 else 'Rural' if x < 1000 else 'Suburban' for x in data['density']]

In [58]:
# Add day of the week
data['Weekday'] = data['date_key'].dt.dayofweek
data['is_weekend'] = (data['date_key'].dt.dayofweek < 5).astype(int)

# Add date, month and year
data['day'] = data['date_key'].dt.day
data['month'] = data['date_key'].dt.month
data['year'] = data['date_key'].dt.year

In [59]:
# Add federal holidays
cal = calendar()
holidays = cal.holidays(start="2017-01-01", end="2019-12-31")

data['is_holiday'] = (data["date_key"].isin(holidays)).astype(int)

In [ ]:
# Add categorical variables for date and zip
data['year'] = pd.Categorical(data['year'])
data['month'] = pd.Categorical(data['month'])
data['day'] = pd.Categorical(data['day'])
data['zip5'] = pd.Categorical(data['zip5'])

In [ ]:
# Add binary variables for state, rural/urban, region, weekday, year, month, and day of week
# dropped variables that were turned into binary
cols = ['state', 'USR', 'Region', 'Weekday', 'year', 'month', 'day']
df = data[cols]
dataDummies = pd.get_dummies(df,drop_first=True)
data = pd.concat([data, dataDummies], axis = 1)
data = data.drop(['state', 'USR', 'Region', 'year', 'month', 'day'], axis = 1)

In [60]:
data

,date_key,zip5,impact_score,grid_lat,grid_lon,Date,Time,ForecastRange,x,y,...,density,state,Region,USR,Weekday,is_weekend,day,month,year,is_holiday
0,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,0.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
1,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,6.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
2,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,12.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
3,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,18.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
4,2017-01-02,2722,16.868994,41.5,-71.0,2017-01-02,0.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,0,1,2,1,2017,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394695,2019-12-30,98421,NaN,47.5,-122.5,2019-12-30,18.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,0,1,30,12,2019,0
394696,2019-12-31,98421,NaN,47.5,-122.5,2019-12-31,0.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0
394697,2019-12-31,98421,NaN,47.5,-122.5,2019-12-31,6.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0
394698,2019-12-31,98421,NaN,47.5,-122.5,2019-12-31,12.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0


## Store full data with missing rows

In [61]:
data.to_hdf("full.h5", key='df', mode='a')

In [62]:
data.shape

(394700, 133)

## Delete redudant columns

In [63]:
complete = data.loc[pd.notna(data["Date"])]
redundant_col = ["Date", 'lat', 'lng']
complete = complete.drop(redundant_col, axis=1)

In [64]:
# Final complete dataframe (no missing rows)
complete

,date_key,zip5,impact_score,grid_lat,grid_lon,Time,ForecastRange,x,y,5_Wave_Geopotential_Height_isobaric,...,density,state,Region,USR,Weekday,is_weekend,day,month,year,is_holiday
0,2017-01-01,2722,20.268081,41.5,-71.0,0.0,0.0,578.0,97.0,5493.184570,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
1,2017-01-01,2722,20.268081,41.5,-71.0,6.0,0.0,578.0,97.0,5514.240723,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
2,2017-01-01,2722,20.268081,41.5,-71.0,12.0,0.0,578.0,97.0,5544.382324,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
3,2017-01-01,2722,20.268081,41.5,-71.0,18.0,0.0,578.0,97.0,5575.850586,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
4,2017-01-02,2722,16.868994,41.5,-71.0,0.0,0.0,578.0,97.0,5614.513184,...,0.000000,MA,New England Northeast,Rural,0,1,2,1,2017,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394695,2019-12-30,98421,NaN,47.5,-122.5,18.0,0.0,950.0,170.0,5661.496094,...,208.237785,WA,Pacific West,Rural,0,1,30,12,2019,0
394696,2019-12-31,98421,NaN,47.5,-122.5,0.0,0.0,950.0,170.0,5667.814453,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0
394697,2019-12-31,98421,NaN,47.5,-122.5,6.0,0.0,950.0,170.0,5667.208984,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0
394698,2019-12-31,98421,NaN,47.5,-122.5,12.0,0.0,950.0,170.0,5655.993652,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0


## Store complete data (without missing rows)

In [65]:
### Save data as hdf5 file
complete.to_hdf("complete.h5", key='df', mode='a')